# Chroma 벡터스토어 튜토리얼

이 노트북에서는 LangChain과 Chroma를 사용하여 벡터스토어를 생성하고 활용하는 방법을 학습합니다.
`Chroma`는 오픈소스 임베딩 데이터베이스로, 문서나 텍스트를 벡터화하여 저장하고 검색할 수 있게 해주는 벡터스토어입니다.

### 벡터스토어의 주요기능
- 문서 임베딩 및 저장
- 유사도 검색 및 필터링
- 영구 저장


In [ ]:
%pip install chromadb

임베딩해서 저장할 문서 준비

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("data/training_log_all.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

split_docs = text_splitter.split_documents(docs)

print(len(split_docs))

print(split_docs[0])

In [ ]:
# 필요한 라이브러리 임포트
from langchain_community.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 설정 (한국어 지원 모델)
embedding_model = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1", 
    model_kwargs={'device': 'cpu'},  # CPU 사용 (GPU가 있다면 'cuda'도 가능)
    encode_kwargs={'normalize_embeddings': True}  # 벡터 정규화 활성화
)

print("라이브러리 및 임베딩 모델 설정 완료")


In [ ]:
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="training_log",
)

 `db.get()` 함수는 Chroma 벡터 DB에 저장된 모든 문서와 메타데이터를 조회하는 함수입니다.

In [ ]:
db.get()

### Chroma의 persist_directory 기능
- `persist_directory`를 사용하면 벡터 DB를 파일 시스템에 영구적으로 저장할 수 있습니다
- 프로그램을 재시작해도 이전에 저장한 데이터를 다시 불러올 수 있습니다

In [ ]:
DB_PATH = "./chroma_db"

persist_db = Chroma.from_documents(
    split_docs, 
    embedding_model, 
    persist_directory=DB_PATH,
    collection_name="training_log"
)

In [ ]:
persist_db.get()

## 2. 유사도 검색 (Similarity Search)

저장된 문서들 중에서 질의와 가장 유사한 문서를 찾는 기능입니다.

- query : 검색할 문장
- k : 반환할 결과의 수
- filter : 메타데이터로 필터링


In [ ]:
# 기본 유사도 검색
query = "2대대의 분대 전술 훈련이 언제 진행되었나요?"

# 가장 유사한 3개 문서 검색
search_results = db.similarity_search(
    query=query,
    k=3  # 상위 3개 결과 반환
)

print(f"검색 질의: '{query}'")
print("검색 결과:")
for i, doc in enumerate(search_results):
    print(f"\n{i+1}. 내용: {doc.page_content}")
    print(f"   메타데이터: {doc.metadata}")


In [ ]:
# 유사도 점수와 함께 검색
query2 = "23일에는 어떤 훈련이 진행되었나요?"

# 유사도 점수와 함께 검색 (점수가 낮을수록 더 유사함)
result = db.similarity_search(
    query=query2,
    k=3
)

print(f"검색 질의: '{query2}'")
print("검색 결과:")
for i, doc in enumerate(search_results):
    print(f"\n{i+1}. 내용: {doc.page_content}")
    print(f"   메타데이터: {doc.metadata}")


### 3. 새로운 문서 추가

기존 벡터스토어에 새로운 문서를 추가할 수 있습니다.


In [ ]:
from langchain_core.documents import Document

# 새로운 문서 추가
new_documents = [
    Document(
        page_content="7월 19일 금요일 식단은 아침에 북어국과 계란말이, 점심에는 치킨마요 덮밥과 단무지, 저녁에는 부대찌개와 김치전이 제공됩니다.",
        metadata={"category":"food","date": "2025-07-19", "day": "Friday"}
    ),
    Document(
        page_content="7월 20일 토요일 식단은 아침에 미역국과 스크램블에그, 점심에는 제육볶음과 콩나물국, 저녁에는 김치찌개와 어묵볶음이 나옵니다.",
        metadata={"category":"food","date": "2025-07-20", "day": "Saturday"}
    ),
    Document(
        page_content="7월 21일 일요일 식단은 아침에 떡국과 김, 점심에는 함박스테이크와 감자조림, 저녁에는 라면과 김밥이 제공됩니다.",
        metadata={"category":"food","date": "2025-07-21", "day": "Sunday"}
    )
]

# 벡터스토어에 새 문서 추가
db.add_documents(new_documents)

print("새로운 문서 2개 추가 완료!")
print(f"현재 총 문서 수: {db._collection.count()}")

# 추가된 문서 검색 테스트
test_query = "20"
test_results = db.similarity_search(test_query, k=3)

print(test_results)


### 4. 메타데이터 필터링

메타데이터를 기반으로 검색 결과를 필터링할 수 있습니다.

In [ ]:
# 메타데이터 필터를 사용한 검색
query3 = "20"

# 중급 난이도 문서만 검색
filtered_results = db.similarity_search(
    query=query3,
    k=5,
    filter={"category": "food"}
)

print("필터링된 검색 결과:")
for i, doc in enumerate(filtered_results):
    print(f"{i+1}. 내용: {doc.page_content}")
    print(f"   카테고리: {doc.metadata['category']}")